In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv("data/AB_NYC_2019.csv")

In [5]:
df = df[['latitude','longitude','price','minimum_nights','number_of_reviews','reviews_per_month','calculated_host_listings_count', 'availability_365']]

## Question 1

In [8]:
df.isnull().sum()

latitude                              0
longitude                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [29]:
print('reviews_per_month', '10052')

reviews_per_month 10052


## Question 2

In [13]:
df.minimum_nights.median() 

3.0

In [15]:
df.minimum_nights.quantile(0.5)

3.0

## Spliting the data

In [76]:
from sklearn.model_selection import train_test_split
X = df.drop(['price'], axis=1)
y = df['price']
y=y.apply(np.log1p)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [77]:
y_train

13575    4.605170
48476    4.060443
44499    4.262680
17382    4.875197
14638    4.709530
           ...   
13198    3.931826
14583    4.836282
6168     5.703782
12248    4.189655
20523    4.532599
Name: price, Length: 29337, dtype: float64

## Question 3

In [78]:
from sklearn.linear_model import LinearRegression

In [79]:
X_train_zero = X_train['reviews_per_month'].fillna(0)
X_train_mean = X_train['reviews_per_month'].fillna(int(X_train['reviews_per_month'].mean()))
X_test_zero = X_test['reviews_per_month'].fillna(0)
X_test_mean = X_test['reviews_per_month'].fillna(int(X_test['reviews_per_month'].mean()))
X_val_zero = X_val['reviews_per_month'].fillna(0)
X_val_mean = X_val['reviews_per_month'].fillna(int(X_val['reviews_per_month'].mean()))

In [80]:
linreg_zero = LinearRegression()
linreg_zero.fit(X_train_zero.values.reshape(-1,1), y_train)


LinearRegression()

In [81]:
pred = linreg_zero.predict(X_test_zero.values.reshape(-1,1))

In [82]:
from sklearn.metrics import mean_squared_error
from math import sqrt
score = sqrt(mean_squared_error(y_test, pred))
print('RMSE: ', round(score, 2) )

RMSE:  0.69


In [83]:
linreg_mean = LinearRegression()
linreg_mean.fit(X_train_mean.values.reshape(-1,1), y_train)

pred = linreg_mean.predict(X_test_mean.values.reshape(-1,1))
score = sqrt(mean_squared_error(y_test, pred))
print('RMSE: ', round(score, 2) )

RMSE:  0.69


## Question 4

In [84]:
rs = [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]


## Question 5

In [85]:
seeds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

rmses = []

for seed in seeds:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=seed)
    X_train = X_train['reviews_per_month'].fillna(0)
    X_test = X_test['reviews_per_month'].fillna(0)
    X_val = X_val['reviews_per_month'].fillna(0)
    linreg = LinearRegression()
    linreg.fit(X_train.values.reshape(-1,1), y_train)
    
    pred = linreg.predict(X_test.values.reshape(-1,1))
    score = sqrt(mean_squared_error(y_test, pred))
    print('RMSE: ', round(score, 2))
    rmses.append(score)

std = np.std(rmses)
print('STD: ',round(std, 3))
      

RMSE:  0.7
RMSE:  0.68
RMSE:  0.7
RMSE:  0.7
RMSE:  0.68
RMSE:  0.71
RMSE:  0.7
RMSE:  0.69
RMSE:  0.69
RMSE:  0.7
STD:  0.008


## Question 6

In [103]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=9)
X_train = pd.DataFrame(X_train['reviews_per_month'].fillna(0))
X_test = pd.DataFrame(X_test['reviews_per_month'].fillna(0))
X_val = pd.DataFrame(X_val['reviews_per_month'].fillna(0))

X_train = pd.concat([X_train, X_val])

y_train = pd.concat([y_train, y_val])
linreg = LinearRegression()
linreg.fit(X_train, y_train)

pred = linreg.predict(X_test)
score = sqrt(mean_squared_error(y_test, pred))
print('RMSE: ', round(score, 2))


RMSE:  0.7


In [104]:
type(X_val)

pandas.core.frame.DataFrame